In [1]:
# best model - 0.518043347
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression

### Logistic Regression
: losgistic regression모델에서 좋은 결과가 많이 나왔고, 그 중 가장 좋은 점수가 나온 모델코드 기입

In [2]:
# Load the data
train_df = pd.read_csv('G:/내 드라이브/3-2AI_4-1deep/chat_gpt_competition/train.csv')
test_df = pd.read_csv('G:/내 드라이브/3-2AI_4-1deep/chat_gpt_competition/test.csv')

# Separate features and target
X_train = train_df['text']
y_train = train_df['label']
X_test = test_df['text']

In [3]:
# Preprocess the text data using TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [4]:
# Split the data into training and validation sets
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [5]:
# Set up the logistic regression model
lr = LogisticRegression(max_iter=5000, C=0.1, penalty='l2', solver='saga', n_jobs=-1)

# Define hyperparameters to search over using GridSearchCV
params = {'C': [0.1, 1, 10], 'penalty': ['l1', 'l2'], 'solver': ['liblinear', 'saga']}

# Perform cross-validation and hyperparameter tuning
grid_search = GridSearchCV(lr, params, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train_split, y_train_split)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 34.5min finished
C:\Users\user\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:1353: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 24.
  warnings.warn("'n_jobs' > 1 does not have any effect when"


GridSearchCV(cv=5,
             estimator=LogisticRegression(C=0.1, max_iter=5000, n_jobs=-1,
                                          solver='saga'),
             n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga']},
             verbose=1)

In [6]:
# Get the best model
best_lr = grid_search.best_estimator_

# Evaluate the performance of the model on the validation set
y_val_pred = best_lr.predict(X_val)
print('Accuracy on validation set:', accuracy_score(y_val, y_val_pred))
print('Classification report:/n', classification_report(y_val, y_val_pred))

# Train the best model on the entire training set
best_lr.fit(X_train, y_train)

# Make predictions on the test set
y_test_pred = best_lr.predict(X_test)

# Save the results to a submission file
submission_df = pd.DataFrame({'id': test_df['id'], 'label': y_test_pred})
submission_df.to_csv('G:/내 드라이브/3-2AI_4-1deep/chat_gpt_competition/submission.csv', index=False)

Accuracy on validation set: 0.8724683544303797
Classification report:/n               precision    recall  f1-score   support

           0       0.86      0.87      0.86      2839
           1       0.92      0.96      0.94      2235
           2       0.82      0.76      0.79      1866
           3       0.89      0.90      0.90      1757
           4       0.88      0.94      0.91       479
           5       0.86      0.77      0.81       204
           6       0.86      0.47      0.61        51
           7       0.72      0.37      0.49        49

    accuracy                           0.87      9480
   macro avg       0.85      0.76      0.79      9480
weighted avg       0.87      0.87      0.87      9480

